In [88]:
#from pulp import *
import numpy as np
import matplotlib.pyplot as plt

In [89]:
#lendo arquivos
arquivo = open("instancia2.txt", 'r')
texto = arquivo.readlines()
arquivo.close()
print(texto)

['8\n', '40\n', '35\n', '12000\n', '45\n', '18\n', '5\n', '10\n', '200\n', '100\n', '440\n', '250\n', '220\n', '350\n', '180\n', '250']


In [90]:
#salvando lista de semanas em T
t = int(texto[0].split()[0])
T = []
for i in range(t):
    T.append(int(i+1))

print("Semanas:",T)

#Salvando preços de C1, C2
CC1 = int(texto[1].split()[0])
CC2 = int(texto[2].split()[0])
print("Custo de compra de C1:", CC1, "\nCusto de compra de C2:", CC2)

#Salvando custo fixo por pedido
CF = int(texto[3].split()[0])
print("Custo fixo por pedido:", CF)

#Salvando custos de estoques
VP1 = int(texto[4].split()[0])
VP2 = int(texto[5].split()[0])
VC1 = int(texto[6].split()[0])
VC2 = int(texto[7].split()[0])
print("VP1 =", VP1, "\nVP2 =", VP2, "\nVC1 =", VC1, "\nVC2 =", VC2)

#Salvando as demandas de P1
d = []

for i in range(8, len(texto)):
    d.append(int(texto[i].split()[0]))

for i in range(t):
    demandas = dict(zip(T, d))
    
print("Demandas:", demandas)

Semanas: [1, 2, 3, 4, 5, 6, 7, 8]
Custo de compra de C1: 40 
Custo de compra de C2: 35
Custo fixo por pedido: 12000
VP1 = 45 
VP2 = 18 
VC1 = 5 
VC2 = 10
Demandas: {1: 200, 2: 100, 3: 440, 4: 250, 5: 220, 6: 350, 7: 180, 8: 250}


In [91]:
#Parametros e dados
demanda = {1:200, 2:100, 3:440, 4:250, 5:220, 6:350, 7:180, 8:250}

In [92]:
#!pip install gurobipy
from gurobipy import *

#modelo
model = Model('MPS')

#criando as variaveis
EP1t = model.addVars(T, vtype=GRB.INTEGER, name = 'EP1t')
EP2t = model.addVars(T, vtype=GRB.INTEGER, name = 'EP2t')
EC1t = model.addVars(T, vtype=GRB.INTEGER, name = 'EC1t')
EC2t = model.addVars(T, vtype=GRB.INTEGER, name = 'EC2t')
P1t = model.addVars(T, vtype=GRB.INTEGER, name = 'P1t')
P2t = model.addVars(T, vtype=GRB.INTEGER, name = 'P2t')
C1t = model.addVars(T, vtype=GRB.INTEGER, name = 'C1t')
C2t = model.addVars(T, vtype=GRB.INTEGER, name = 'C2t')
CFt = model.addVars(T, vtype=GRB.BINARY, name = 'CFt')

In [93]:
#definindo a função objetivo
som1c1 = quicksum(CC1*C1t[i] for i in T)
som1c2 = quicksum(CC2*C2t[i] for i in T)
som1 = som1c1 + som1c2

som2EP1t = quicksum(VP1*EP1t[i] for i in T)
som2EP2t = quicksum(VP2*EP2t[i] for i in T)
som2 = som2EP1t + som2EP2t

som3EC1t = quicksum(VC1*EC1t[i] for i in T)
som3EC2t = quicksum(VC2*EC2t[i] for i in T)
som3 = som3EC1t + som3EC2t

z = som1 + som2 + som3
model.setObjective(z, GRB.MINIMIZE)

In [94]:
#definindo as restrições
model.addConstrs(EP1t[i]==EP1t[i-1]+P1t[i]+demanda[i] for i in T if i!=1)
model.addConstrs(EP2t[i]==EP2t[i-1]+P2t[i]-(demanda[i]*2) for i in T if i!=1)
model.addConstrs(EC1t[i]==EC1t[i-1]+C1t[i]-P2t[i]-(demanda[i]*3) for i in T if i!=1)
model.addConstrs(EC2t[i]==EC2t[i-1]+C2t[i]-(P2t[i]*2) for i in T if i!=1)

model.addConstrs((C1t[i]+C2t[i])+CF <= CFt[i]*1000000 for i in T)
model.addConstrs(P1t[i]>=demanda[i]-EP1t[i] for i in T)
model.addConstrs(P2t[i]>=(demanda[i]*2)-EP2t[i] for i in T)
model.addConstrs(C1t[i]>=P2t[i]+(demanda[i]*2)-EC1t[i] for i in T)
model.addConstrs(C2t[i]>=(P2t[i]*2)-EC2t[i] for i in T)

#resolver:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 68 rows, 72 columns and 202 nonzeros
Model fingerprint: 0xd9268ae2
Variable types: 0 continuous, 72 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+04]
Found heuristic solution: objective 892050.00000
Presolve removed 18 rows and 26 columns
Presolve time: 0.00s
Presolved: 50 rows, 46 columns, 150 nonzeros
Variable types: 0 continuous, 46 integer (0 binary)

Root relaxation: objective 6.967150e+05, 39 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    696715.00000 696715.000  0.00%     -    0s

Explored 0 nodes (39 simplex iterations) in 0.06 seconds
Thread

In [95]:
#display
model.display()

Minimize
   <gurobi.LinExpr: 45.0 EP1t[1] + 45.0 EP1t[2] + 45.0 EP1t[3] + 45.0 EP1t[4] + 45.0 EP1t[5] + 45.0 EP1t[6] + 45.0 EP1t[7] + 45.0 EP1t[8] + 18.0 EP2t[1] + 18.0 EP2t[2] + 18.0 EP2t[3] + 18.0 EP2t[4] + 18.0 EP2t[5] + 18.0 EP2t[6] + 18.0 EP2t[7] + 18.0 EP2t[8] + 5.0 EC1t[1] + 5.0 EC1t[2] + 5.0 EC1t[3] + 5.0 EC1t[4] + 5.0 EC1t[5] + 5.0 EC1t[6] + 5.0 EC1t[7] + 5.0 EC1t[8] + 10.0 EC2t[1] + 10.0 EC2t[2] + 10.0 EC2t[3] + 10.0 EC2t[4] + 10.0 EC2t[5] + 10.0 EC2t[6] + 10.0 EC2t[7] + 10.0 EC2t[8] + 40.0 C1t[1] + 40.0 C1t[2] + 40.0 C1t[3] + 40.0 C1t[4] + 40.0 C1t[5] + 40.0 C1t[6] + 40.0 C1t[7] + 40.0 C1t[8] + 35.0 C2t[1] + 35.0 C2t[2] + 35.0 C2t[3] + 35.0 C2t[4] + 35.0 C2t[5] + 35.0 C2t[6] + 35.0 C2t[7] + 35.0 C2t[8]>
Subject To
   R0 : <gurobi.LinExpr: -1.0 EP1t[1] + EP1t[2] + -1.0 P1t[2]> = 100.0
   R1 : <gurobi.LinExpr: -1.0 EP1t[2] + EP1t[3] + -1.0 P1t[3]> = 440.0
   R2 : <gurobi.LinExpr: -1.0 EP1t[3] + EP1t[4] + -1.0 P1t[4]> = 250.0
   R3 : <gurobi.LinExpr: -1.0 EP1t[4] + EP1t[5] + -1

In [96]:
print("Valor da função objetivo: "+str(round(model.ObjVal, 2)))
for v in model.getVars():
    print(str(v.varName)+" = "+str(round(v.x,2)))

Valor da função objetivo: 696715.0
EP1t[1] = 0.0
EP1t[2] = 100.0
EP1t[3] = 540.0
EP1t[4] = 790.0
EP1t[5] = 1010.0
EP1t[6] = 1360.0
EP1t[7] = 1540.0
EP1t[8] = 1790.0
EP2t[1] = 2460.0
EP2t[2] = 2260.0
EP2t[3] = 1380.0
EP2t[4] = 880.0
EP2t[5] = 440.0
EP2t[6] = 220.0
EP2t[7] = 110.0
EP2t[8] = 55.0
EC1t[1] = 5740.0
EC1t[2] = 5440.0
EC1t[3] = 4120.0
EC1t[4] = 3370.0
EC1t[5] = 2710.0
EC1t[6] = 1180.0
EC1t[7] = 500.0
EC1t[8] = 125.0
EC2t[1] = 0.0
EC2t[2] = 0.0
EC2t[3] = 0.0
EC2t[4] = 0.0
EC2t[5] = -0.0
EC2t[6] = -0.0
EC2t[7] = 0.0
EC2t[8] = -0.0
P1t[1] = 200.0
P1t[2] = -0.0
P1t[3] = 0.0
P1t[4] = 0.0
P1t[5] = 0.0
P1t[6] = 0.0
P1t[7] = 0.0
P1t[8] = -0.0
P2t[1] = -0.0
P2t[2] = -0.0
P2t[3] = -0.0
P2t[4] = 0.0
P2t[5] = 0.0
P2t[6] = 480.0
P2t[7] = 250.0
P2t[8] = 445.0
C1t[1] = -0.0
C1t[2] = -0.0
C1t[3] = -0.0
C1t[4] = -0.0
C1t[5] = -0.0
C1t[6] = -0.0
C1t[7] = 110.0
C1t[8] = 820.0
C2t[1] = -0.0
C2t[2] = -0.0
C2t[3] = -0.0
C2t[4] = -0.0
C2t[5] = -0.0
C2t[6] = 960.0
C2t[7] = 500.0
C2t[8] = 890.0
CFt[1]